# 第4回課題
内包表記，並列処理

In [27]:
#!pip install joblib

In [28]:
from joblib import Parallel, delayed
import numpy as np
import pprint
pp = pprint.PrettyPrinter(indent=4)

## 内包表記
python における高速化テクの1つである内包表記の実装（for 文の削減）

### 課題3

以下の for 文を内包表記に書き直しなさい．
```[python]
data = []
for i in range(5):
    for j in range(4):
        data.append(i*j)
data
```
出力：
[0, 0, 0, 0, 0, 1, 2, 3, 0, 2, 4, 6, 0, 3, 6, 9, 0, 4, 8, 12]

In [20]:
data=[x*y for x in range(5)for y in range(4)]
data

[0, 0, 0, 0, 0, 1, 2, 3, 0, 2, 4, 6, 0, 3, 6, 9, 0, 4, 8, 12]

### 課題2

以下の for 文を内包表記に書き直しなさい．
```[python]
data = []
for i in range(5):
    inner = []
    for j in range(i, 6):
        inner.append(i)
    data.append(inner)
data
```
出力：
[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3], [4, 4]]

In [21]:
data =[[i for j in range(i, 6)]for i in range(5)]
data

[[0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1], [2, 2, 2, 2], [3, 3, 3], [4, 4]]

## 並列処理
python における高速化テクの1つである並列化の実装（複数コア，CPU の使用）

joblib だけでなく multiprocessing もある

### 例：2乗の場合

In [22]:
%%timeit 3 # 3回下記のコードを実行した時の時間を表示
r = []
for i in range(10000):
    r.append(i ** 2)
r[:3] + r[-3:]

100 loops, best of 5: 2.92 ms per loop


In [29]:
?Parallel

In [34]:
%%timeit 3 
power = lambda x: x**2
r = Parallel(n_jobs=-1, verbose=0)([delayed(power)(i) for i in range(10000)])
r[:3] + r[-3:]

1 loop, best of 5: 251 ms per loop


In [ ]:
power = lambda x: x**2
r = Parallel(n_jobs=-1, verbose=3)( [delayed(power)(i) for i in range(10000)] )
r[:3] + r[-3:]

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 2032 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:    0.2s finished


[0, 1, 4, 99940009, 99960004, 99980001]

In [31]:
%%timeit 3
# numpy が最速というのはよくある
np.arange(10000) ** 2

The slowest run took 381.62 times longer than the fastest. This could mean that an intermediate result is being cached.
100000 loops, best of 5: 13.9 µs per loop


In [32]:
%%timeit 3
np.fromiter(range(10000), dtype=np.int) ** 2

1000 loops, best of 5: 400 µs per loop


### 課題1

x = [(i, i+1) for i in range(100000)] としたとき，各 index 毎に積を求めよ

出力 [0, 2, 6,（中略） 9999500006, 9999700002, 9999900000]

In [37]:
seki = lambda x: x*(x+1)
r = Parallel(n_jobs=-1, verbose=0)([delayed(seki)(i) for i in range(100000)])
r[:3] + r[-3:]

[0, 2, 6, 9999500006, 9999700002, 9999900000]

### 発展課題
C = np.cumsum(np.arange(1000)) の各項の差を求めよ

出力[1, 2, 3,（中略） 997, 998, 999] = np.arange(1, 1000)

In [56]:
sa = lambda x,y: y-x
r = Parallel(n_jobs=-1, verbose=0)([delayed(sa)(np.cumsum(np.arange(i))[i-1],np.cumsum(np.arange(i+1))[i]) for i in range(1,1000)])
r[:3] + r[-3:]

[1, 2, 3, 997, 998, 999]